In [9]:
# Load your data
# link of data: http://datosabiertos.stps.gob.mx/Datos/DIL/clave/Tasa_de_Desocupacion.csv
# url = "http://datosabiertos.stps.gob.mx/Datos/DIL/clave/Tasa_de_Desocupacion.csv"
#!pip install voila
#!pip install dash pandas plotly
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import plotly.express as px
import dash
from dash.dependencies import Input, Output
from dash import dcc, html
import dash_html_components as html 
url='https://raw.githubusercontent.com/Jordan-Villanueva/Dashboard_Veredis/main/Tasa_de_Desocupacion.csv'
data = pd.read_csv(url, encoding='latin-1')
data = data[(data['Entidad_Federativa'] != 'Nacional')].reset_index(drop=True)
data = data.drop(columns=['Unnamed: 7', 'Unnamed: 8'])

# Create the Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("Población Económicamente Activa en México"),

    dcc.Dropdown(
        id='year-dropdown',
        options=[
            {'label': str(year), 'value': year} for year in data['Periodo'].unique()
        ],
        value=data['Periodo'].max(),
        multi=False,
        style={'width': '50%'}
    ),

    dcc.Dropdown(
        id='trimester-dropdown',
        options=[
            {'label': str(trimestre), 'value': trimestre} for trimestre in data['Trimestre'].unique()
        ],
        value=data['Trimestre'].max(),
        multi=False,
        style={'width': '50%'}
    ),

    html.Div(
        id='message',
        children='',
        className='centered-message'
    ),

    dcc.Graph(id='bar-chart'),
])

# ... (remaining code)


# Define callback to update bar chart
@app.callback(
    [Output('bar-chart', 'figure'),
     Output('message', 'children')],
    [Input('year-dropdown', 'value'),
     Input('trimester-dropdown', 'value')],
    prevent_initial_call=True
)
def update_chart(selected_year, selected_trimester):
    filtered_data = data[(data['Periodo'] == selected_year) & (data['Trimestre'] == selected_trimester)]
    if not filtered_data.empty:
        fig = px.bar(
            filtered_data,
            x='Entidad_Federativa',
            y='Poblacion_Economicamente_Activa',
            color='Sexo',
            barmode='group',
            labels={'Poblacion_Economicamente_Activa': 'Población (millones de habitantes)'},
            color_discrete_sequence = ['steelblue','magenta'],
            title=f'Población Económicamente Activa en {selected_year} - Trimestre {selected_trimester}'
        )
        fig.update_xaxes(tickangle=-60)

        return fig, None
    else:
        message = 'No data available for the selected period. Sorry'
        return {},message

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
